# Import Libraries

In [1]:
from swarmintelligence import GeneticAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/GA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_gens': 100, 'mr': 0.2, 'cr': 0.6}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    gwo_g = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    gwo_b = GeneticAlgorithm(
                        k=threshold,
                        pop_size=optimal_params['pop_size'],
                        max_gens=optimal_params['max_gens'],
                        fitness_function=fitness_function,
                        mr=optimal_params['mr'],
                        cr=optimal_params['cr'],
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
926.8554247925402  |  2434.858216347896  |  1688.3879284601205
876.2184774387262  |  2413.143815265621  |  1781.8879286568795
926.0975422335869  |  2423.753635954644  |  1755.258485695116
923.6957561039942  |  2422.8291815370917  |  1781.4200742224416
928.2502713786567  |  2431.5628606031037  |  1787.2999067097403
915.4845524215262  |  2408.982869081599  |  1747.870717242586
915.1858456255154  |  2433.4300352634  |  1760.2829738005348
925.5221766938911  |  2421.5849342731644  |  1784.2748626792916
929.8212431609678  |  2383.022680871266  |  1777.911000794155
899.3960023071234  |  2409.1689539933795  |  1771.3999311126706
923.8761817806651  |  2409.7457140366837  |  1786.6021639067221
921.7245192026212  |  2422.3504549300133  |  1781.1323988216438
901.7866695300447  |  2428.4203933604936  |  1782.9217452274356
925.0913064965829  |  2435.8717286502574  |  1784.8658112388891
924.7553234982364  |  2420.474931350251  |  1749.6862939494747
913.451775119814

980.2509521378896  |  2363.4187803512805  |  2158.5156630729516
946.7980097627221  |  2360.5894363988905  |  2087.5654902438937
952.3219047592188  |  2304.315913531894  |  2173.1645727764426
1003.0571680994952  |  2390.886196710193  |  2147.5867454085633
989.9480845866556  |  2330.239612760439  |  2170.9295398073227
1002.8658339338574  |  2369.7729426177666  |  2159.3395768703094
1002.8241351323218  |  2385.2775842878164  |  2151.2363387320493
982.825808421826  |  2290.321002018933  |  2180.028975968109
974.7538013343751  |  2369.5629005541323  |  2072.7667756662204
1004.7326101372672  |  2385.2862941987987  |  2176.7030902118636
995.7075914446583  |  2376.1579200436972  |  2175.9866898355776
996.1207865131378  |  2374.7086131436427  |  2117.6396518245915
980.5429743698832  |  2360.4991484175416  |  2097.968016872011
979.553346565637  |  2385.267936865118  |  2140.094558522172
988.9915998471936  |  2281.393294973078  |  2107.7449564625344
945.5047968338363  |  2388.81900187474  |  2175

2922.6091404921626  |  2940.212167173074  |  2942.1808044726376
2942.4212357109973  |  2897.9994877568006  |  2901.8971386218495
2868.9404925696954  |  2888.5453621910533  |  2911.0787189004964
2857.5596452120994  |  2871.5239803852073  |  2839.9790389246036
2946.4992893555914  |  2922.1299461012995  |  2895.811348476117
2862.551137000334  |  2852.8993325021097  |  2940.7952692391627
2900.67329498869  |  2909.5453735974534  |  2922.679931932803
2934.32591784428  |  2884.8618622856243  |  2939.8631355419484
2924.5678579904634  |  2848.447782620197  |  2930.734918172512
2922.291988696726  |  2764.061812629131  |  2932.448351726599
2928.3642916862264  |  2898.7978613904115  |  2909.24121239121
2886.57031896949  |  2796.9089137258284  |  2936.729180133126
2897.501538663236  |  2901.9648033137746  |  2930.835518909313
2919.2232592674236  |  2808.330413275359  |  2936.3438928433625
2920.6675164036906  |  2832.0138865560166  |  2944.6424373599857
2937.212602893089  |  2932.4984761938076  |  2

3333.092614197654  |  2004.2539278992492  |  2626.9626784732673
3262.5274551732828  |  2002.0531597218617  |  2633.6441707815034
3301.8507577580526  |  1994.8143801353822  |  2628.7275243767194
3305.3398861705455  |  1887.7685820835281  |  2629.103283694317
3253.4358857876446  |  1881.4129097331438  |  2609.887008683836
3297.0381184975568  |  2001.2089702817882  |  2623.7160471160814
3327.161996162536  |  1997.460173639628  |  2635.475173962518
3274.386437125807  |  1910.6247725561407  |  2623.062136350338
3338.978155125436  |  1966.3905455179838  |  2562.4249368405917
3353.925619571221  |  2004.4945853679887  |  2572.451209395709
3345.7889731512996  |  1966.7836403636982  |  2633.4795376457255
3358.6464828062453  |  1980.743357878238  |  2620.9921236337696
3352.4572695698253  |  1996.9756248457413  |  2628.7275243767194
3357.6292010600146  |  1974.8049427808264  |  2629.3250204855826
3347.1129843642275  |  1993.223505641054  |  2557.440612018255
3334.803685706833  |  1968.147628806462

1645.4749998945654  |  5160.6808351724085  |  1708.134005584855
1614.8704626643857  |  5166.840981432672  |  1750.5974501413139
1640.6919235399205  |  5198.768336280531  |  1717.7103706068165
1655.4859014721412  |  5192.135086815402  |  1651.1196544122968
1591.916548364834  |  5175.99042240464  |  1746.7364815197038
1640.8823398418965  |  5179.075139971578  |  1745.984309906772
1571.2004581703268  |  5180.217727518626  |  1738.4117520340096
1654.4389066102763  |  5201.724514414529  |  1667.603280905404
1643.7774731653353  |  5167.991117693002  |  1710.3284424289159
1609.1618725031274  |  5181.095156807154  |  1726.089223291738
1651.2623330259353  |  5132.420458426648  |  1611.6681213118404
1649.5121515885294  |  5184.082150543054  |  1759.1359980441362
1621.593753223142  |  5203.063095199314  |  1752.476796948567
1632.3633293074495  |  5181.323952007959  |  1709.969257817072
1644.0521730671921  |  5168.384411767014  |  1734.6191584993153
1627.6588381682202  |  5203.802604932915  |  174

5622.459735792904  |  5580.793193195853  |  1583.965219930753
5637.8686417530225  |  5635.939007733248  |  1564.676078568968
5599.759404824816  |  5606.912055342753  |  1582.6016056873978
5638.702881770834  |  5635.68771779448  |  1587.5133112505773
5613.91583614589  |  5501.514092152087  |  1482.8234913380297
5569.879980097106  |  5594.613715745656  |  1574.984068171184
5634.166910749508  |  5636.411887253815  |  1540.9606118710008
5636.310106328227  |  5617.12274917627  |  1566.909303698703
5636.354203297744  |  5616.182436403133  |  1532.162056416606
5640.428003063228  |  5621.352890233272  |  1586.5759897203664
5601.8786605627465  |  5611.501001945039  |  1529.0576062744772
5605.104117636489  |  5601.479103156371  |  1554.381029936626
5636.802829798272  |  5621.280216593843  |  1537.560869330301
5562.792065418708  |  5616.14951014036  |  1580.3646379769025
5622.831191420481  |  5623.6746764850195  |  1589.6638813278805
5579.225601081218  |  5570.746966915954  |  1587.173288894977
5

## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[126, 178]","[99, 172]","[125, 181]",919.050428,2419.378529,1771.365406,...,"[[125, 255, 255, 255, 255, 255, 255, 255, 255,...","[926.8554247925402, 876.2184774387262, 926.097...","[2434.858216347896, 2413.143815265621, 2423.75...","[1688.3879284601205, 1781.8879286568795, 1755....","[2.936565399169922, 2.91542911529541, 2.998245...","[2496.5556831359863, 2233.885809580485, 2091.2...","[49.96554495986195, 47.26400120155386, 45.7300...","[14.157391038719332, 14.640193915227757, 14.92...","[0.7657218971785226, 0.7537807629623878, 0.746...","[0.9375641866981371, 0.9405975617849287, 0.943..."
1,AirplaneF16.tiff,3,otsu,max,"[104, 148, 188]","[79, 135, 183]","[118, 162, 192]",956.487348,2507.081282,1836.677975,...,"[[118, 255, 255, 255, 255, 255, 255, 255, 255,...","[943.8778787591925, 963.3580718850623, 942.605...","[2507.4600823240207, 2508.9415268304765, 2510....","[1845.4196619875336, 1853.6665045040677, 1808....","[3.218118906021118, 3.2632012367248535, 3.1723...","[1981.639762878418, 1967.474142710368, 1700.96...","[44.51561257444873, 44.356218760286225, 41.242...","[15.160556528041457, 15.19171327461711, 15.823...","[0.7880802184521457, 0.7826136108342002, 0.738...","[0.9512957986293796, 0.9507985145790435, 0.953..."
2,AirplaneF16.tiff,4,otsu,max,"[86, 131, 163, 194]","[55, 107, 151, 192]","[99, 142, 175, 196]",973.031370,2549.019660,1867.015617,...,"[[99, 255, 255, 255, 255, 255, 255, 196, 255, ...","[978.9736730020303, 961.7850362163794, 977.246...","[2522.5316489595957, 2537.875060070551, 2519.6...","[1863.202080198731, 1864.6977225699225, 1844.4...","[3.1723520755767822, 3.0362513065338135, 3.999...","[1633.1291058858235, 1430.1655286153157, 1794....","[40.41199210489163, 37.81752938275206, 42.3569...","[16.00059841925823, 16.576940548686746, 15.592...","[0.7548378525829579, 0.7158593893396831, 0.730...","[0.9585589049479225, 0.9622655880613508, 0.954..."
3,AirplaneF16.tiff,5,otsu,max,"[66, 112, 145, 175, 201]","[47, 90, 130, 168, 198]","[85, 127, 160, 182, 200]",987.545727,2582.318842,1889.701853,...,"[[127, 255, 200, 255, 255, 255, 200, 200, 200,...","[981.6206236853918, 989.2984754375746, 978.457...","[2581.56564060861, 2558.232339436722, 2572.272...","[1892.743853777131, 1898.0847446084792, 1893.8...","[3.9625720977783203, 3.0606918334960938, 3.073...","[1470.4434000651042, 1303.072193145752, 940.93...","[38.346360975522884, 36.09809126734753, 30.674...","[16.456320484562937, 16.98111883592548, 18.395...","[0.7377549787010316, 0.748529629802508, 0.7765...","[0.9626747562470487, 0.9652826213089609, 0.975..."
4,Lena.png,2,otsu,max,"[136, 193]","[76, 144]","[95, 141]",984.405546,2356.466902,2151.059567,...,"[[141, 141, 141, 141, 141, 141, 141, 141, 141,...","[980.2509521378896, 946.7980097627221, 952.321...","[2363.4187803512805, 2360.5894363988905, 2304....","[2158.5156630729516, 2087.5654902438937, 2173....","[3.5724241733551025, 2.9080564975738525, 3.769...","[2200.27455774943, 2576.27553431193, 1977.9296...","[46.9070843023677, 50.75702448244903, 44.47392...","[14.7060348392038, 14.020880516242356, 15.1686...","[0.6301471959120393, 0.5998717790848122, 0.624...","[0.888260002577956, 0.8787457812315557, 0.8782..."
5,Lena.png,3,otsu,max,"[122, 169, 208]","[62, 110, 162]","[83, 114, 151]",1044.728886,2541.442820,2232.839133,...,"[[151, 151, 151, 151, 151, 151, 151, 151, 151,...","[1009.9122411434287, 1043.1517639362244, 1038....","[2568.96501475383, 2588.8587199820468, 2570.26...","[2251.1126710778744, 2240.6188393938655, 2262....","[3.745478391647339, 3.454881429672241, 3.01074...","[1705.4722315470378, 1094.7696215311687, 1558....","[41.297363493896775, 33.08730302595195, 39.473...","[15.812357082379915, 17.73757623083814, 16.204...","[0.65148017745

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/GA/GA_otsu_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/GA/GA_otsu_RGB_dict.pkl', 'wb'))